In [7]:
from df_func.make_XY import Words_Matrix
from etl_func.etl_data import transform_stock_df
import pandas as pd
import os
from datetime import date
rawdata_path = '/Users/alexlo/Desktop/Project/MLEM_Final/rawdata'
workdata_path = '/Users/alexlo/Desktop/Project/MLEM_Final/workdata'
data_time = tuple([date(2019,1,1), date(2021,12,31)])
day_arg = 2 
cutoff_arg = 0.03
company = '聯電'
stock_code = '2303'

os.chdir(rawdata_path)
with open('stopwords_zh.txt', 'r') as file:
    stop_words = file.read().splitlines()

os.chdir(workdata_path)
word_df = pd.read_parquet(company + '_word_df.parquet')
stock_df = pd.read_parquet(stock_code + '_stock_df.parquet')
stock_df = transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg)

words_matrix = Words_Matrix(word_df, stock_df, data_time, stop_words)
X = words_matrix.feature_X_byChi2(k=1000)
Y = words_matrix.Y_matrix
print(X.head())
print(Y.head())

/Users/alexlo/Desktop/Project/MLEM_Final/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


    上修   上億  上周三  上如果  上揚美系  上櫃日  上漲次  上漲除了   上船  下半年在  ...  鴻海聚積聯電  鴻海聯發科統一  \
0  0.0  0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...     0.0      0.0   
1  0.0  0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...     0.0      0.0   
2  0.0  0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...     0.0      0.0   
3  0.0  0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...     0.0      0.0   
4  0.0  0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...     0.0      0.0   

   麥格理證券  黃育仁   默默   點從   點跌   鼎鑫   齊開  龐大緊繃度  
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  

[5 rows x 1000 columns]
0    False
1    False
2    False
3    False
4    False
Name: Label, dtype: bool


In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

classifier_list = {
    'kNN': KNeighborsClassifier,
    'Ridge': RidgeClassifier,
    'Desision Tree': DecisionTreeClassifier,
    'Random Forest': RandomForestClassifier,
    'Gradient Boosting': GradientBoostingClassifier,
    'MLP': MLPClassifier,
}
for classifier in classifier_list:
    clf = classifier_list[classifier]()
    scores = cross_val_score(clf, X , Y, cv = 5)
    print(classifier, ':', round(scores.mean(), 3))
    print(scores)

kNN : 0.8639255793711239
[0.86593407 0.86483516 0.86373626 0.86373626 0.86138614]
Ridge : 0.8641460849381641
[0.86483516 0.86373626 0.86373626 0.86373626 0.86468647]
Desision Tree : 0.8351312274084552
[0.82307692 0.82967033 0.82967033 0.84615385 0.84708471]
Random Forest : 0.856011557199676
[0.85494505 0.86483516 0.85054945 0.85714286 0.85258526]
Gradient Boosting : 0.8599690518502401
[0.85714286 0.86153846 0.85934066 0.86263736 0.85918592]
MLP : 0.8718396015425718
[0.87032967 0.87802198 0.87252747 0.86813187 0.87018702]


In [22]:
print(classifier, ':', round(scores.mean(), 3))


MLP : 0.872


In [24]:
for day_arg in np.array(0,5):
    print(day_arg)

1
5


In [18]:
import numpy as np
from tqdm import tqdm
all_results = dict()
for day in ([1, 2, 3]):
    for cut in tqdm([0, 0.01, 0.02, 0.03]):
        
        results = np.random.binomial(10, 0.8, 10)
        keys = f'day{day}cut{cut}'
        all_results[keys] = results

100%|██████████| 4/4 [00:00<00:00, 24385.49it/s]


In [23]:
import pandas as pd
pd.DataFrame(all_results)

,day1cut0,day1cut0.01,day1cut0.02,day1cut0.03,day2cut0,day2cut0.01,day2cut0.02,day2cut0.03,day3cut0,day3cut0.01,day3cut0.02,day3cut0.03
0,9,6,7,6,8,8,9,9,5,8,7,8
1,5,8,10,10,10,7,6,10,8,8,9,7
2,8,6,7,8,10,9,6,8,8,8,9,7
3,9,6,10,7,7,8,9,5,9,9,9,7
4,9,7,8,10,8,9,7,9,9,9,9,8
5,10,6,7,8,8,10,8,8,10,8,8,7
6,6,9,5,7,9,10,5,9,8,8,10,9
7,9,7,8,10,9,6,7,9,5,9,8,10
8,7,9,7,10,8,7,10,9,5,6,7,8
9,8,9,8,9,9,8,8,9,8,9,7,7


In [24]:
import os
import pandas as pd
os.chdir('/Users/alexlo/Desktop/Project/MLEM_Final/workdata')
x = pd.read_csv('測試.csv')